### Structured Streaming
**`Micro-batch | Checkpointing | Streaming -> Delta`**


### What Is Structured Streaming?
- Structured Streaming is Spark's built-in stream processing engine. The key insight from the official Databricks docs: although it is called streaming, it works on micro-batch processing. At regular intervals, a batch of new data is ingested, optionally transformed, and written to a sink like a Delta table.

The genius of the API is that it looks almost identical to a regular batch DataFrame. The same filter(), groupBy(), join(), withColumn() transformations work on both. `The only difference is readStream instead of read, and writeStream instead of write.`


| **Concept**           | **Batch (Days 1-3)**                  | **Structured Streaming **                     |
|-----------------------|---------------------------------------|------------------------------------------------------|
| **Read**              | spark.read.format('delta').load(path) | spark.readStream.format('delta').table(name)         |
| **Transform**         | df.filter(), df.groupBy()             | Same — identical transformation API                  |
| **Write**             | df.write.saveAsTable()                | df.writeStream.toTable() or .start()                 |
| **Execution**         | Runs once, finishes, stops            | Runs continuously — processes new data as it arrives |
| **Trigger**           | No trigger — manual execution         | Micro-batch, fixed interval, or availableNow         |
| **Progress tracking** | No tracking needed                    | Checkpoint folder tracks every batch processed       |



> **`From official Databricks docs: Delta Lake is deeply integrated with Spark Structured Streaming through readStream and writeStream. Delta Lake overcomes many limitations typically associated with streaming systems — including coalescing small files from low-latency ingest, maintaining exactly-once processing, and efficiently discovering which files are new.`**

### Micro-batch — How It Actually Works
> Micro-batch is the default execution model for Structured Streaming in Spark.   
Instead of processing one event at a time (true streaming), Spark collects all new data that arrived since the last batch, processes it together, and writes the result. This repeats on a trigger interval.


### MICRO-BATCH CYCLE:

> `Trigger fires -> Spark checks source for new data -> Collects new records into one batch
-> Applies transformations -> Writes results to sink (Delta table)
-> Updates checkpoint -> Sleeps until next trigger -> Repeat`

### `Trigger Modes — Four Options`

| **Trigger**               | **Code**                              | **Behaviour**                                                                                           | **Best For**                                 |
|---------------------------|---------------------------------------|---------------------------------------------------------------------------------------------------------|----------------------------------------------|
| **Default (unspecified)** | .writeStream.start()                  | Runs next micro-batch as soon as previous one finishes. No gap.                                         | Maximum throughput — continuous processing   |
| **Fixed interval**        | .trigger(processingTime='30 seconds') | Waits 30 seconds between each micro-batch regardless of how fast previous finished.                     | Predictable latency — dashboards, monitoring |
| **availableNow**          | .trigger(availableNow=True)           | Processes ALL currently available data then stops. Like a batch job that uses streaming infrastructure. | Daily batch jobs, testing, backfill          |
| **Once (deprecated)**     | .trigger(once=True)                   | Processes one micro-batch then stops. Replaced by availableNow.                                         | Legacy — use availableNow instead            |



### Checkpointing — The Memory of a Stream
A checkpoint is a folder on disk that Structured Streaming writes to after every micro-batch. It records exactly which data has already been processed. When the stream restarts after a failure or notebook restart, it reads the checkpoint and resumes exactly where it left off — not from the beginning.

**`Without a checkpoint:`** every time the stream restarts it reprocesses all data from the beginning. This causes duplicate records in your output Delta table.

**`With a checkpoint:`** restart is safe. Spark reads the checkpoint, skips already-processed data, and continues from the last committed batch. This gives you exactly-once processing guarantees.


**What Is Inside the Checkpoint Folder?**
> ```
/Volumes/ecommerce/.../checkpoints/events_stream/
  commits/          <- one file per completed micro-batch (batch 0, batch 1, batch 2...)
  offsets/          <- tracks which source data was included in each batch
  metadata          <- stream identity and configuration
  state/            <- stateful operation state (for aggregations with watermarks)```


**`Critical rule from Databricks docs:`** Every streaming writer must have a UNIQUE checkpoint location. If two streams share the same checkpoint, they corrupt each other's state and you get unpredictable results.

**Safe storage location:** Store checkpoints inside your Volume path, not in /tmp/. Volumes are persistent across cluster restarts. /tmp/ is wiped when the cluster terminates — losing your checkpoint means losing stream progress.

| Scenario                                  | What Happens                                                                                   |
|-------------------------------------------|------------------------------------------------------------------------------------------------|
| First run — no checkpoint exists          | Stream starts from the beginning of the source data. Creates the checkpoint folder.            |
| Stream runs successfully                  | After each micro-batch, checkpoint is updated with the batch ID and source offsets.            |
| Cluster restarts or notebook re-runs      | Stream reads checkpoint, skips already-processed batches, resumes from last committed offset.  |
| Source has no new data                    | Stream sees nothing new in checkpoint vs source. Processes zero records. Checkpoint unchanged. |
| You change the stream query significantly | Must delete checkpoint and restart — checkpoint is tied to the specific query structure.       |
| You delete the checkpoint manually        | Stream resets completely — reprocesses all source data from the beginning.                     |


> `For writing events to a Delta table: always use append mode (the default). Complete mode rewrites the entire table every batch — extremely expensive at scale. Update mode requires stateful aggregations with watermarking.`

**Reading a Stream from a Delta Table
Since your ecommerce events are already in a Delta table** (ecommerce.bronze.events from Day 2), you can use that as your streaming source directly. Delta Lake tracks which rows have been added and serves only new ones to each micro-batch.

**`How Delta knows what is new:`** Every write to a Delta table creates a new entry in the _delta_log/ transaction log with a version number. The streaming reader tracks which version it last processed in the checkpoint. The next batch reads only the versions it has not seen yet.

**`Important constraint:`** Structured Streaming from a Delta table only works if the source table is append-only. If the source table has UPDATE or DELETE operations, use skipChangeCommits=true option (recommended by Databricks for Runtime 12.2+) or handle via Change Data Feed.


### Batch vs Streaming — When to Use Which

| Factor              | Batch (your Days 1-3)                                         | Structured Streaming (Day 4)                                     |
|---------------------|---------------------------------------------------------------|------------------------------------------------------------------|
| Data freshness      | Hours or days old — processed on schedule                     | Seconds to minutes — processed as it arrives                     |
| Complexity          | Simple — one run, one result                                  | More complex — checkpoint management, trigger tuning             |
| Cost                | Lower — cluster runs only during job                          | Higher — cluster runs continuously for true streaming            |
| Good for            | Daily feature tables, monthly reports, large historical loads | Real-time dashboards, fraud detection, live recommendation feeds |
| Your ecommerce case | Rebuilding user_features_gold daily                           | Ingesting live purchase events to Bronze in near real-time       |


## Practical Tasks

**Simulate Streaming from Delta Table**

**`What We Are Doing`**
> Read from ecommerce.bronze.events_br as a streaming source. We use `readStream instead of read`. The schema must be defined explicitly for streaming reads from files — but since we are reading from a Delta table, Spark infers the schema automatically from the Delta log.


In [0]:
# Define paths (store in variables, never hardcode in stream calls)

CATALOG         = 'ecommerce'
SOURCE_TABLE    = f'{CATALOG}.bronze.events_br'
OUTPUT_TABLE    = f'{CATALOG}.bronze.events_stream'
CHECKPOINT_PATH = '/Volumes/ecommerce/sc_ecommerce/vol_ecommerce/checkpoints/events_stream/'

print(f'Source:     {SOURCE_TABLE}')
print(f'Output:     {OUTPUT_TABLE}')
print(f'Checkpoint: {CHECKPOINT_PATH}')


Source:     ecommerce.bronze.events_br
Output:     ecommerce.bronze.events_stream
Checkpoint: /Volumes/ecommerce/sc_ecommerce/vol_ecommerce/checkpoints/events_stream/


In [0]:
# Create a streaming DataFrame from the Bronze Delta table
# readStream instead of read — this is the ONLY change vs batch reading

df_stream = spark.readStream \
    .format('delta') \
    .table(SOURCE_TABLE)

# Check that it is a streaming DataFrame
print(f'Is streaming: {df_stream.isStreaming}')   # should print: True
print(f'Schema:')
df_stream.printSchema()


Is streaming: True
Schema:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- _ingested_at: timestamp (nullable = true)
 |-- _run_date: string (nullable = true)



`df_stream.isStreaming = True` means we now have a streaming DataFrame.

we cannot call .count() or .show() on it directly — those are batch actions.
> To see data from a streaming DataFrame, you must either:
  - (a) write it to a sink with writeStream, OR
  - (b) call display(df_stream) in a Databricks notebook — this starts a live stream display.

In [0]:
# lets Apply transformations (same API as batch)
# Add a processing timestamp so we know when each record was picked up by the stream
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when

df_transformed = df_stream \
    .withColumn('_stream_processed_at', F.current_timestamp()) \
    .withColumn('event_date', F.to_date(col('event_time'))) \
    .filter(col('event_type').isin(['view', 'cart', 'purchase']))  # same filter as Day 2

print('Transformations defined (lazy — nothing has run yet)')
print(f'Is still streaming: {df_transformed.isStreaming}')  # still True


Transformations defined (lazy — nothing has run yet)
Is still streaming: True


### Task 2
**Write Streaming Output to Delta Table**
**`What We Are Doing`**
- Write the transformed streaming DataFrame to a new Delta table using writeStream. We use trigger(availableNow=True) so the stream processes all existing data and stops cleanly — no need to manually interrupt it. This is the recommended mode for notebook-based streaming in Databricks.

In [0]:
# Write streaming output to Delta using availableNow trigger
# availableNow: processes all currently available data then stops automatically

query = df_transformed.writeStream \
    .format('delta') \
    .outputMode('append') \
    .option('checkpointLocation', CHECKPOINT_PATH) \
    .trigger(availableNow=True) \
    .toTable(OUTPUT_TABLE)

# Wait for the stream to finish (needed with availableNow)
query.awaitTermination()
print('Stream complete!')
print(f'Output written to: {OUTPUT_TABLE}')

Stream complete!
Output written to: ecommerce.bronze.events_stream


**`What each option does:`**

`outputMode('append')`     -> only write NEW rows each batch, never modify existing ones   
`checkpointLocation `      -> folder where Spark saves batch progress — REQUIRED   
`trigger(availableNow=True)` -> process all data now then stop (safe for notebooks)   
`toTable(OUTPUT_TABLE) `   -> write to Delta table by name (creates it if not exists)   
`awaitTermination() `      -> block the cell until the stream finishes (needed for availableNow)

![image_1772163793468.png](./image_1772163793468.png "image_1772163793468.png")

In [0]:
# let' sVerify the output table was created and has data
spark.sql(f'DESCRIBE DETAIL {OUTPUT_TABLE}') \
    .select('numFiles', 'sizeInBytes') \
    .show()

record_count = spark.table(OUTPUT_TABLE).count()
print(f'Records written to stream output table: {record_count:,}')

# Should match Bronze count (minus any filtered records)
bronze_count = spark.table(SOURCE_TABLE).count()
print(f'Bronze source records: {bronze_count:,}')


+--------+-----------+
|numFiles|sizeInBytes|
+--------+-----------+
|      20| 1954959245|
+--------+-----------+

Records written to stream output table: 109,950,743
Bronze source records: 109,950,743


### TASK 3 — Simulate New Data Arriving (Streaming Behaviour)
**`What We Are Doing`**

To see streaming in action — not just a one-time batch — we append new rows to the source Bronze table and re-run the stream. Because the checkpoint remembers what was already processed, the stream picks up ONLY the new rows. This is the core streaming behaviour.


In [0]:
# Now lte's Append new simulated events to the Bronze source table
# This simulates new data arriving from a live source (Kafka, IoT, app events etc.)

from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.types import *


schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", IntegerType(), True),   # 👈 INT
    StructField("category_id", LongType(), True),     # 👈 MUST be Long (value is huge)
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", IntegerType(), True),      # 👈 INT
    StructField("user_session", StringType(), True),
    StructField("_ingested_at", TimestampType(), True),
    StructField("_run_date", StringType(), True),
])

new_events = spark.createDataFrame([
    (datetime(2019, 12, 1, 10, 0, 0), 'purchase',
     9999001, 2053013555631882655,
     'electronics.smartphone', 'samsung',
     799.99, 111111111,
     'new-session-001', datetime.now(), '2019-12-01'),

    (datetime(2019, 12, 1, 10, 1, 0), 'view',
     9999002, 2053013555631882655,
     'electronics.smartphone', 'apple',
     1099.99, 222222222,
     'new-session-002', datetime.now(), '2019-12-01'),

    (datetime(2019, 12, 1, 10, 2, 0), 'cart',
     9999003, 2053013566100866035,
     'appliances.kitchen', 'lg',
     349.99, 333333333,
     'new-session-003', datetime.now(), '2019-12-01'),
], schema)


# Append to Bronze — this is the new data the stream will pick up
new_events.write \
    .format('delta') \
    .mode('append') \
    .saveAsTable(SOURCE_TABLE)

print(f'3 new events appended to {SOURCE_TABLE}')
print(f'New Bronze total: {spark.table(SOURCE_TABLE).count():,}')


3 new events appended to ecommerce.bronze.events_br
New Bronze total: 109,950,746


In [0]:
# Re-run the stream — it should pick up ONLY the 3 new rows
# The checkpoint remembers all previous batches — no reprocessing of old data

query2 = df_transformed.writeStream \
    .format('delta') \
    .outputMode('append') \
    .option('checkpointLocation', CHECKPOINT_PATH) \
    .trigger(availableNow=True) \
    .toTable(OUTPUT_TABLE)

query2.awaitTermination()

# Verify only the 3 new rows were added
new_total = spark.table(OUTPUT_TABLE).count()
print(f'Output table now has: {new_total:,} records')
print('The difference should be exactly 3 — the new rows only')


Output table now has: 109,950,746 records
The difference should be exactly 3 — the new rows only


**`This is the power of checkpointing. The stream ran twice. First run: processed all existing Bronze data. Second run: processed ONLY the 3 new rows. No duplicates. No reprocessing. This is exactly-once delivery.`**

### TASK 4 — Query Streaming Results
`What We Are Doing`
>Once data is written to the output Delta table, you query it like any other Delta table using regular batch reads. The streaming part is only for writing — querying is always batch. This is one of the key advantages of Delta Lake as a streaming sink.


In [0]:
# Basic query on the streaming output table
df_out = spark.table(OUTPUT_TABLE)

print('=== STREAMING OUTPUT TABLE — OVERVIEW ===')
display(df_out.limit(10))

=== STREAMING OUTPUT TABLE — OVERVIEW ===


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,_ingested_at,_run_date,_stream_processed_at,event_date
2019-10-04T08:50:52.000Z,view,7901061,2053013556487520725,furniture.kitchen.chair,null,51.22,533182637,a4da7c6b-0c06-4944-bb43-d8f8e86d0b3f,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,1307073,2053013558920217191,computers.notebook,acer,720.48,515158111,39d25f67-79f1-4945-9bdb-5f434505cfc1,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,1700954,2053013553031414015,computers.peripherals.monitor,samsung,223.68,514032452,b6401e78-7207-4846-b8c9-9ba0314a3db0,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,5100182,2053013553375346967,null,moov,51.41,531767955,45720368-8e8a-40c4-b8cd-84e441e3ff49,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,1004872,2053013555631882655,electronics.smartphone,samsung,292.08,556652631,26db4f41-880d-42e9-b443-38bc8fa93008,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,1201216,2172371436436455782,electronics.tablet,lenovo,253.43,545382331,fbdbbf91-9230-4f1f-a95a-69dbf266aed3,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,28101291,2053013564918072245,null,confetti,7.72,528253546,d8f99f9e-b345-4354-98f3-1fe456b4ffcc,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,1004838,2053013555631882655,electronics.smartphone,oppo,179.28,512523731,76c02347-65db-417f-ad06-c9af1cb0bae6,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,3600661,2053013563810775923,appliances.kitchen.washer,samsung,296.53,514191743,ff26f202-cb64-4135-bf01-a9cfb6a9c4fe,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04
2019-10-04T08:50:53.000Z,view,21400921,2053013561579406073,electronics.clocks,casio,600.53,512438486,ba1cc954-1561-4682-a850-d79ee4018607,2026-02-26T03:31:01.056Z,2026-02-25,2026-02-27T03:38:55.049Z,2019-10-04


In [0]:
#  Event type distribution in streamed data
print('=== EVENT TYPE DISTRIBUTION ===')
df_out.groupBy('event_type') \
    .count() \
    .orderBy('count', ascending=False) \
    .show()

=== EVENT TYPE DISTRIBUTION ===
+----------+---------+
|event_type|    count|
+----------+---------+
|      view|104335510|
|      cart|  3955447|
|  purchase|  1659789|
+----------+---------+



In [0]:
# Show the new December rows to confirm they were streamed separately
print('=== NEWLY STREAMED ROWS (December 2019) ===')
display(
    df_out.filter(F.col('event_date') == '2019-12-01')
)


=== NEWLY STREAMED ROWS (December 2019) ===


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,_ingested_at,_run_date,_stream_processed_at,event_date
2019-12-01T10:00:00.000Z,purchase,9999001,2053013555631882655,electronics.smartphone,samsung,799.99,111111111,new-session-001,2026-02-27T03:59:24.556Z,2019-12-01,2026-02-27T04:01:21.971Z,2019-12-01
2019-12-01T10:01:00.000Z,view,9999002,2053013555631882655,electronics.smartphone,apple,1099.99,222222222,new-session-002,2026-02-27T03:59:24.556Z,2019-12-01,2026-02-27T04:01:21.971Z,2019-12-01
2019-12-01T10:02:00.000Z,cart,9999003,2053013566100866035,appliances.kitchen,lg,349.99,333333333,new-session-003,2026-02-27T03:59:24.556Z,2019-12-01,2026-02-27T04:01:21.971Z,2019-12-01


In [0]:
# Check the stream processing timestamps
# _stream_processed_at shows WHEN each record was picked up by the stream
# You will see two distinct timestamps — one for batch 1, one for batch 2
print('=== STREAM PROCESSING TIMESTAMPS ===')
df_out.select('event_date', '_stream_processed_at') \
    .groupBy('event_date', '_stream_processed_at') \
    .count() \
    .orderBy('_stream_processed_at', ascending=False) \
    .show(truncate=False)


=== STREAM PROCESSING TIMESTAMPS ===
+----------+-----------------------+-------+
|event_date|_stream_processed_at   |count  |
+----------+-----------------------+-------+
|2019-12-01|2026-02-27 04:01:21.971|3      |
|2019-11-22|2026-02-27 03:38:55.049|1568243|
|2019-11-17|2026-02-27 03:38:55.049|6395377|
|2019-11-25|2026-02-27 03:38:55.049|1593582|
|2019-11-19|2026-02-27 03:38:55.049|1728541|
|2019-11-23|2026-02-27 03:38:55.049|1561716|
|2019-11-20|2026-02-27 03:38:55.049|1700086|
|2019-11-14|2026-02-27 03:38:55.049|3069726|
|2019-11-12|2026-02-27 03:38:55.049|1987569|
|2019-10-30|2026-02-27 03:38:55.049|1210145|
|2019-10-12|2026-02-27 03:38:55.049|1479896|
|2019-10-31|2026-02-27 03:38:55.049|1245479|
|2019-10-13|2026-02-27 03:38:55.049|1639071|
|2019-11-16|2026-02-27 03:38:55.049|6502957|
|2019-11-13|2026-02-27 03:38:55.049|2019165|
|2019-10-29|2026-02-27 03:38:55.049|1227910|
|2019-10-14|2026-02-27 03:38:55.049|1454702|
|2019-11-18|2026-02-27 03:38:55.049|2021512|
|2019-10-15|2026-0

In [0]:

# Use DESCRIBE HISTORY to see each streaming batch as a Delta version
# Every micro-batch that wrote - data creates a new version in the Delta log
spark.sql(f'DESCRIBE HISTORY {OUTPUT_TABLE}').show(10, truncate=False)


+-------+-------------------+----------------+--------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------------------------------------+------------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------+------------+--------------------------------------------------+
|version|timestamp          |userId          |userName            |operation       |operationParameters                                                                     

In [0]:
print(f'Is still streaming: {df_transformed.isStreaming}') 

Is still streaming: True


In [0]:
spark.streams.active

[]

### Live Stream Display in Notebook (Bonus)
In a Databricks notebook, calling `display()` on a streaming DataFrame starts a live streaming job with a real-time progress dashboard. The stream runs continuously until you manually stop it by interrupting the cell. Use this for monitoring and exploration — NOT for production writes.


In [0]:
# Live display of the stream in notebook (runs continuously)
# WARNING: This starts a continuous stream — interrupt the cell to stop it
# Do NOT use this for production writes — use writeStream.toTable() instead
path = CHECKPOINT_PATH
print(path)
# display(df_transformed)
display(df_transformed, checkpointLocation = path)

# In the notebook UI you will see:
# - A live table updating with each micro-batch
# - A streaming progress dashboard showing:
#     inputRowsPerSecond  — how fast data is arriving
#     processedRowsPerSecond — how fast Spark is processing
#     batchId — which micro-batch is currently running
#     numInputRows — rows in the current batch


/Volumes/ecommerce/sc_ecommerce/vol_ecommerce/checkpoints/events_stream/
Checkpointing to /Volumes/ecommerce/sc_ecommerce/vol_ecommerce/checkpoints/events_stream/


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6949988642392912>, line 7
      5 print(path)
      6 # display(df_transformed)
----> 7 display(df_transformed, checkpointLocation = path)

File /databricks/python_shell/lib/dbruntime/display.py:133, in Display.display(self, input, *args, **kwargs)
    131     pass
    132 elif self._cf_helper is not None and isinstance(input, ConnectDataFrame):
--> 133     self.display_connect_table(input, **kwargs)
    134 elif isinstance(input, ConnectDataFrame):
    135     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:94, in Display.display_connect_table(self, df, **kwargs)
     89     raise type(
     90         e
     91     )("IPython shell encountered an error or was missing data, please restart the notebook or contact Databricks support"
     92       ) from e
     93 if df.isStreaming:
---> 94    